This is an experimental (stub...) Schema.org dashboard using Jupyter/iPython

In [2]:
# Import libraries

import unittest
import os
import pprint
from os import path, getenv
from os.path import expanduser
import logging # https://docs.python.org/2/library/logging.html#logging-levels
import glob
import argparse
import StringIO
import sys

print os.getcwd()
if os.getcwd().endswith('schemaorg'):
    print "Working from: ", os.getcwd()
else:
    #print "cd .."
    os.chdir("..")

# We'll need our lib/ dir, plus AppEngine's files:
sys.path.append( os.getcwd() ) 
sdk_path = getenv('APP_ENGINE',  expanduser("~") + '/google-cloud-sdk/platform/google_appengine/')
sys.path.insert(0, sdk_path) 
print "GAE: ", sdk_path

# previous useful:
import dev_appserver
dev_appserver.fix_sys_path()
# pprint.pprint(os.environ.copy())

/Users/danbri/sdo/official/schemaorg/scripts
GAE:  /Users/danbri/google-cloud-sdk/platform/google_appengine/


In [3]:
import rdflib
from rdflib import Graph
from rdflib import RDF, RDFS
from rdflib.term import URIRef, Literal
from rdflib.parser import Parser
from rdflib.serializer import Serializer
from rdflib.plugins.sparql import prepareQuery
from rdflib.compare import graph_diff
import threading
from api import inLayer,read_file, full_path, read_schemas, read_extensions, read_examples, namespaces, DataCache, getMasterStore
from api import  setInTestHarness, GetAllTypes, Unit, GetImmediateSubtypes # old API
from apirdflib import getNss



In [4]:
# Setup
setInTestHarness(True)
import sdoapp

rdflib.plugin.register("json-ld", Serializer, "rdflib_jsonld.serializer", "JsonLDSerializer")

store = getMasterStore()
read_schemas(loadExtensions=True)
read_extensions(sdoapp.ENABLED_EXTENSIONS)
graphs = list(store.graphs())

In [5]:
# Convert SPARQL results to Pandas DataFrame:
from pandas import DataFrame

def sparql2df(a):
    c = []
    for b in a.bindings:
        rowvals=[]
        for k in a.vars:
            rowvals.append(b[k])
        c.append(rowvals)

    df = DataFrame(c)
    df.columns = [str(v) for v in a.vars]
    return df

# Overview

We have two APIs, the original pseudo-RDF unit/node structure, plus also now RDFLib. 
It is better to use the latter as it gives access to SPARQL, parsers/serializers etc.

In [54]:
for g in graphs:
  print g, "\n"
    
#print graphs[7], len(graphs[7])
sdocore = graphs[8]
print sdocore

[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'IOMemory']]. 

<http://health-lifesci.schema.org/> a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'IOMemory']. 

[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'IOMemory']]. 

<http://attic.schema.org/> a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'IOMemory']. 

[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'IOMemory']]. 

[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'IOMemory']]. 

<http://meta.schema.org/> a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'IOMemory']. 

<http://auto.schema.org/> a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'IOMemory']. 

<http://schema.org/> a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'IOMemory']. 

[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'IOMemory']]. 

<http://bib.schema.org/> a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'IOMemory']. 

<http://pending.schema.org/> a rdfg:Graph;rdflib:storage [a

In [55]:
pprint.pprint( "Found %s types." % len( GetAllTypes() ) )

'Found 583 types.'


In [56]:
for t in ["Article", "CreativeWork", "Product", "MedicalEntity", "Event"]: 
    someType = Unit.GetUnit(t)
    p_subtypes = GetImmediateSubtypes(someType)
    pprint.pprint( "Direct subtypes of %s: %s" % ( someType.id, ', '.join([str(x.id) for x in p_subtypes]) ) )
 

u'Direct subtypes of Article: NewsArticle, Report, ScholarlyArticle, SocialMediaPosting, TechArticle'
u'Direct subtypes of CreativeWork: Article, Blog, Book, Clip, Code, Comment, Conversation, CreativeWorkSeason, CreativeWorkSeries, DataCatalog, Dataset, DigitalDocument, Episode, Game, Map, MediaObject, Message, Movie, MusicComposition, MusicPlaylist, MusicRecording, Painting, Photograph, PublicationIssue, PublicationVolume, Question, Recipe, Review, Sculpture, Season, Series, SoftwareApplication, SoftwareSourceCode, TVSeason, TVSeries, VisualArtwork, WebPage, WebPageElement, WebSite'
u'Direct subtypes of Product: IndividualProduct, ProductModel, SomeProducts, Vehicle'
u'Direct subtypes of MedicalEntity: '
u'Direct subtypes of Event: BusinessEvent, ChildrensEvent, ComedyEvent, DanceEvent, DeliveryEvent, EducationEvent, ExhibitionEvent, Festival, FoodEvent, LiteraryEvent, MusicEvent, PublicationEvent, SaleEvent, ScreeningEvent, SocialEvent, SportsEvent, TheaterEvent, UserInteraction, Vi

In [57]:
renamed =  sdocore.query("select ?x ?y where { ?x <http://schema.org/supersededBy> ?y }")

for (old, new) in renamed:
    print "older: %s -> newer: %s" % (old, new)

older: http://schema.org/episodes -> newer: http://schema.org/episode
older: http://schema.org/ingredients -> newer: http://schema.org/recipeIngredient
older: http://schema.org/serviceArea -> newer: http://schema.org/areaServed
older: http://schema.org/UserPlays -> newer: http://schema.org/InteractionCounter
older: http://schema.org/UserPageVisits -> newer: http://schema.org/InteractionCounter
older: http://schema.org/paymentDue -> newer: http://schema.org/paymentDueDate
older: http://schema.org/colleagues -> newer: http://schema.org/colleague
older: http://schema.org/Taxi -> newer: http://schema.org/TaxiService
older: http://schema.org/UserTweets -> newer: http://schema.org/InteractionCounter
older: http://schema.org/spatial -> newer: http://schema.org/spatialCoverage
older: http://schema.org/significantLinks -> newer: http://schema.org/significantLink
older: http://schema.org/DatedMoneySpecification -> newer: http://schema.org/MonetaryAmount
older: http://schema.org/UserPlusOnes -> n

In [58]:
a = sdocore.query("select ?x ?p ?y where { ?x ?p ?y }")


In [59]:
sparql2df( sdocore.query("select ?item ?type where { ?item a ?type }") )

,item,type
0,http://schema.org/PeopleAudience,http://www.w3.org/2000/01/rdf-schema#Class
1,http://schema.org/CheckOutAction,http://www.w3.org/2000/01/rdf-schema#Class
2,http://schema.org/predecessorOf,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...
3,http://schema.org/VeganDiet,http://schema.org/RestrictedDiet
4,http://schema.org/vehicleModelDate,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...
5,http://schema.org/LakeBodyOfWater,http://www.w3.org/2000/01/rdf-schema#Class
6,http://schema.org/publishingPrinciples,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...
7,http://schema.org/validThrough,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...
8,http://schema.org/Ticket,http://www.w3.org/2000/01/rdf-schema#Class
9,http://schema.org/broadcastServiceTier,http://www.w3.org/1999/02/22-rdf-syntax-ns#Pro...


In [67]:
sparql2df( sdocore.query("select ?x where { ?x rdfs:subClassOf <http://schema.org/Event> } LIMIT 30 ") )


,x
0,http://schema.org/MusicEvent
1,http://schema.org/BusinessEvent
2,http://schema.org/ComedyEvent
3,http://schema.org/FoodEvent
4,http://schema.org/ChildrensEvent
5,http://schema.org/DanceEvent
6,http://schema.org/SaleEvent
7,http://schema.org/ScreeningEvent
8,http://schema.org/SportsEvent
9,http://schema.org/LiteraryEvent


In [68]:
sdocore

<Graph identifier=http://schema.org/ (<class 'rdflib.graph.Graph'>)>

In [71]:
s = "select ?x where { ?x rdfs:subClassOf <http://schema.org/CreativeWork> } LIMIT 60 "
a = sdocore.query(s)
sparql2df( a )

,x
0,http://schema.org/WebPageElement
1,http://schema.org/Map
2,http://schema.org/Series
3,http://schema.org/TVSeason
4,http://schema.org/MusicComposition
5,http://schema.org/CreativeWorkSeries
6,http://schema.org/Movie
7,http://schema.org/PublicationVolume
8,http://schema.org/Game
9,http://schema.org/Blog


In [29]:
# Unit Tests
# Seems we could run them here, https://amodernstory.com/2015/06/28/running-unittests-in-the-ipython-notebook/
# unittest.TextTestRunner().run(suite)